In [1]:
import pandas as pd
from collections import Counter
from statistics import mode, StatisticsError

In [2]:
# Data
Patch_TST = [
    {'stock': 'AAPL', 'model': 'finbert', 'params': {'dropout': 0.47755413461177565, 'batch_size': 128, 'learning_rate': 0.001398397397652222, 'patch_len': 16, 'stride': 10}, 'mse': 16.224550247192383},
    {'stock': 'AAPL', 'model': 'vader', 'params': {'dropout': 0.05258165313452709, 'batch_size': 128, 'learning_rate': 0.001396374190788974, 'patch_len': 48, 'stride': 8}, 'mse': 15.809932708740234},
    {'stock': 'AAPL', 'model': 'roberta', 'params': {'dropout': 0.32987331164742256, 'batch_size': 512, 'learning_rate': 0.0032859510885658713, 'patch_len': 32, 'stride': 10}, 'mse': 15.982783317565918},
    {'stock': 'META', 'model': 'finbert', 'params': {'dropout': 0.10788654461218597, 'batch_size': 512, 'learning_rate': 0.00021461228257735928, 'patch_len': 48, 'stride': 10}, 'mse': 4.389112949371338},
    {'stock': 'META', 'model': 'vader', 'params': {'dropout': 0.08714383148378746, 'batch_size': 128, 'learning_rate': 0.00011693733231826586, 'patch_len': 48, 'stride': 10}, 'mse': 3.926825761795044},
    {'stock': 'META', 'model': 'roberta', 'params': {'dropout': 0.2340746258608002, 'batch_size': 512, 'learning_rate': 6.317125336296553e-05, 'patch_len': 48, 'stride': 8}, 'mse': 4.42146110534668},
    {'stock': 'MSFT', 'model': 'finbert', 'params': {'dropout': 0.1342157557976283, 'batch_size': 256, 'learning_rate': 0.0010699044885843206, 'patch_len': 16, 'stride': 8}, 'mse': 14.091763496398926},
    {'stock': 'MSFT', 'model': 'vader', 'params': {'dropout': 0.3100044627695442, 'batch_size': 512, 'learning_rate': 0.009853374191323868, 'patch_len': 16, 'stride': 8}, 'mse': 13.696978569030762},
    {'stock': 'MSFT', 'model': 'roberta', 'params': {'dropout': 0.49674728207597396, 'batch_size': 512, 'learning_rate': 0.0007384190301642808, 'patch_len': 16, 'stride': 12}, 'mse': 14.181476593017578},
    {'stock': 'NVDA', 'model': 'finbert', 'params': {'dropout': 0.39444384427852586, 'batch_size': 256, 'learning_rate': 0.0011232469935980858, 'patch_len': 16, 'stride': 8}, 'mse': 80.82823181152344},
    {'stock': 'NVDA', 'model': 'vader', 'params': {'dropout': 0.2867828029404956, 'batch_size': 128, 'learning_rate': 0.0005011123349339654, 'patch_len': 16, 'stride': 8}, 'mse': 78.71142578125},
    {'stock': 'NVDA', 'model': 'roberta', 'params': {'dropout': 0.35671718704587974, 'batch_size': 128, 'learning_rate': 0.0004966115296405134, 'patch_len': 32, 'stride': 12}, 'mse': 80.11370086669922},
    {'stock': 'TSLA', 'model': 'finbert', 'params': {'dropout': 0.4036608833994088, 'batch_size': 128, 'learning_rate': 0.0024153010849653983, 'patch_len': 32, 'stride': 12}, 'mse': 108.84541320800781},
    {'stock': 'TSLA', 'model': 'vader', 'params': {'dropout': 0.46726853183654726, 'batch_size': 128, 'learning_rate': 2.2295785327843577e-05, 'patch_len': 48, 'stride': 8}, 'mse': 111.01953887939453},
    {'stock': 'TSLA', 'model': 'roberta', 'params': {'dropout': 0.4840856539410029, 'batch_size': 128, 'learning_rate': 0.00018039359532792313, 'patch_len': 32, 'stride': 8}, 'mse': 110.76618194580078},
    {'stock': 'GOOGL', 'model': 'finbert', 'params': {'dropout': 0.10729936727797121, 'batch_size': 512, 'learning_rate': 0.00021489711725324436, 'patch_len': 48, 'stride': 10}, 'mse': 11.859642028808594},
    {'stock': 'GOOGL', 'model': 'vader', 'params': {'dropout': 0.3479686027278116, 'batch_size': 128, 'learning_rate': 0.0003171779603543097, 'patch_len': 16, 'stride': 10}, 'mse': 11.220919609069824},
    {'stock': 'GOOGL', 'model': 'roberta', 'params': {'dropout': 0.387052709349633, 'batch_size': 128, 'learning_rate': 0.0003691486455250093, 'patch_len': 32, 'stride': 12}, 'mse': 11.966643333435059},
    {'stock': 'AMZN', 'model': 'finbert', 'params': {'dropout': 0.05262276448528757, 'batch_size': 128, 'learning_rate': 7.087859488700552e-05, 'patch_len': 16, 'stride': 8}, 'mse': 16.11032485961914},
    {'stock': 'AMZN', 'model': 'vader', 'params': {'dropout': 0.2707966517803455, 'batch_size': 128, 'learning_rate': 0.0003178160080107326, 'patch_len': 32, 'stride': 12}, 'mse': 15.829183578491211},
    {'stock': 'AMZN', 'model': 'roberta', 'params': {'dropout': 0.23577841437927932, 'batch_size': 512, 'learning_rate': 0.00011373979307051219, 'patch_len': 32, 'stride': 12}, 'mse': 16.378124237060547},
]

Informer_setting = [
    {'stock': 'AAPL', 'model': 'finbert', 'params': {'dropout': 0.3016087404467096, 'batch_size': 128, 'learning_rate': 3.601136968452322e-05, 'label_len': 45}, 'mse': 16.635095596313477},
    {'stock': 'AAPL', 'model': 'vader', 'params': {'dropout': 0.3967571218410976, 'batch_size': 128, 'learning_rate': 2.634242135125069e-05, 'label_len': 30}, 'mse': 16.82978630065918},
    {'stock': 'AAPL', 'model': 'roberta', 'params': {'dropout': 0.22592245160076901, 'batch_size': 256, 'learning_rate': 8.085873490084855e-05, 'label_len': 45}, 'mse': 16.265132904052734},
    {'stock': 'META', 'model': 'finbert', 'params': {'dropout': 0.12302715303437517, 'batch_size': 512, 'learning_rate': 3.2004961268184505e-05, 'label_len': 30}, 'mse': 5.739016056060791},
    {'stock': 'META', 'model': 'vader', 'params': {'dropout': 0.21814121826749155, 'batch_size': 256, 'learning_rate': 7.074036559665999e-05, 'label_len': 15}, 'mse': 5.513366222381592},
    {'stock': 'META', 'model': 'roberta', 'params': {'dropout': 0.49468005981364127, 'batch_size': 256, 'learning_rate': 5.160679096886255e-05, 'label_len': 15}, 'mse': 5.604220390319824},
    {'stock': 'MSFT', 'model': 'finbert', 'params': {'dropout': 0.36912519474853295, 'batch_size': 128, 'learning_rate': 0.00021583236619237555, 'label_len': 30}, 'mse': 14.474627494812012},
    {'stock': 'MSFT', 'model': 'vader', 'params': {'dropout': 0.276481262484511, 'batch_size': 256, 'learning_rate': 6.23610297306941e-05, 'label_len': 15}, 'mse': 14.255833625793457},
    {'stock': 'MSFT', 'model': 'roberta', 'params': {'dropout': 0.11620034897299875, 'batch_size': 256, 'learning_rate': 0.00015869238862200932, 'label_len': 15}, 'mse': 14.622986793518066},
    {'stock': 'NVDA', 'model': 'finbert', 'params': {'dropout': 0.24331447120811894, 'batch_size': 512, 'learning_rate': 0.00023360579748050496, 'label_len': 15}, 'mse': 85.03646087646484},
    {'stock': 'NVDA', 'model': 'vader', 'params': {'dropout': 0.4483936022298001, 'batch_size': 128, 'learning_rate': 0.0001343734200558095, 'label_len': 45}, 'mse': 84.4102783203125},
    {'stock': 'NVDA', 'model': 'roberta', 'params': {'dropout': 0.208118450655918, 'batch_size': 256, 'learning_rate': 8.874267987924277e-05, 'label_len': 15}, 'mse': 85.04058074951172},
    {'stock': 'TSLA', 'model': 'finbert', 'params': {'dropout': 0.48354896156550725, 'batch_size': 128, 'learning_rate': 0.0002654004258020519, 'label_len': 45}, 'mse': 110.26556396484375},
    {'stock': 'TSLA', 'model': 'vader', 'params': {'dropout': 0.45739376058830095, 'batch_size': 256, 'learning_rate': 0.0038129638877524106, 'label_len': 45}, 'mse': 109.8934097290039},
    {'stock': 'TSLA', 'model': 'roberta', 'params': {'dropout': 0.3325276851373125, 'batch_size': 512, 'learning_rate': 0.00016744548935649305, 'label_len': 30}, 'mse': 110.05938720703125},
    {'stock': 'GOOGL', 'model': 'finbert', 'params': {'dropout': 0.42652345379855167, 'batch_size': 128, 'learning_rate': 0.0003624451722715322, 'label_len': 30}, 'mse': 13.93571662902832},
    {'stock': 'GOOGL', 'model': 'vader', 'params': {'dropout': 0.4874677870403265, 'batch_size': 512, 'learning_rate': 0.00024698334481318754, 'label_len': 45}, 'mse': 13.635431289672852},
    {'stock': 'GOOGL', 'model': 'roberta', 'params': {'dropout': 0.1204561161113281, 'batch_size': 512, 'learning_rate': 0.0036543491948239576, 'label_len': 15}, 'mse': 13.97333812713623},
    {'stock': 'AMZN', 'model': 'finbert', 'params': {'dropout': 0.388785700627255, 'batch_size': 512, 'learning_rate': 0.0003248461280468332, 'label_len': 30}, 'mse': 18.973413467407227},
    {'stock': 'AMZN', 'model': 'vader', 'params': {'dropout': 0.17089677774978468, 'batch_size': 256, 'learning_rate': 2.0684070546736584e-05, 'label_len': 15}, 'mse': 18.692325592041016},
    {'stock': 'AMZN', 'model': 'roberta', 'params': {'dropout': 0.41520935909097095, 'batch_size': 512, 'learning_rate': 3.273190009731464e-05, 'label_len': 15}, 'mse': 18.937419891357422},
]

# Convert to DataFrames
patchtst_df = pd.DataFrame([{
    'dropout':d['params']['dropout'],
    'batch_size': d['params']['batch_size'],
    'learning_rate':d['params']['learning_rate'],
    'patch_len': d['params']['patch_len'],
    'stride': d['params']['stride']
} for d in Patch_TST])

informer_df = pd.DataFrame([{
    'dropout':d['params']['dropout'],
    'batch_size': d['params']['batch_size'],
    'learning_rate':d['params']['learning_rate'],
    'label_len': d['params']['label_len']
} for d in Informer_setting])

In [3]:
# Helper function for mode
def safe_mode(series):
    try:
        return mode(series)
    except StatisticsError:
        return Counter(series).most_common(1)[0][0]


In [4]:
# Calculate modes
patchtst_modes = {
    'dropout': safe_mode(patchtst_df['dropout']),
    'batch_size': safe_mode(patchtst_df['batch_size']),
    'learning_rate': safe_mode(patchtst_df['learning_rate']),
    'patch_len': safe_mode(patchtst_df['patch_len']),
    'stride': safe_mode(patchtst_df['stride'])
}

informer_modes = {
    'dropout': safe_mode(informer_df['dropout']),
    'batch_size': safe_mode(informer_df['batch_size']),
    'learning_rate': safe_mode(informer_df['learning_rate']),
    'label_len': safe_mode(informer_df['label_len'])
}


In [6]:
patchtst_modes

{'dropout': 0.47755413461177565,
 'batch_size': 128,
 'learning_rate': 0.001398397397652222,
 'patch_len': 16,
 'stride': 8}

In [7]:
informer_modes

{'dropout': 0.3016087404467096,
 'batch_size': 256,
 'learning_rate': 3.601136968452322e-05,
 'label_len': 15}